This notebook explains how to generate the datasets used for the "Scenario 1" of the proposed benchmark.

In [1]:
# for the moment the directory should be set to the parent to be able to use the LIPS package
import os
os.chdir(os.path.pardir)

In [2]:
path_benchmark = os.path.join("reference_data")
if not os.path.exists(path_benchmark):
    os.mkdir(path_benchmark)

# LIPS : Learning Industrial physical simulationbenchmark suite: the power grid case

The benchmarking platform (as it can be observed in the Figure below) is composed of three main modules: Data, Benchmarking and Evaluation. As the names suggest, each module is specialized with its own set of functions for a specific task. The Data module is the one that helps to generate different dataset and distributions providing a high flexibility using a set of parameters. Once the datasets are generated, the benchmarking platform will take as entry the generated dataset and perform a required benchmark on it. Each benchmark could be precised via the inputs and outputs required for modeling and prediction. Finally, in order to evaluate the performance of each benchmark, the Evaluation module provides a set of functions to inspect the models from different point of views (physics compliances, machine learning metrics, adaptability and readiness.  

The figure below presents the scheme of benchmarking platform which is composed of three distinct parts : 
* DataSet: Generate some dataset for training and evaluation 
* Benchmark: coordinates between different parts and allows train and evaluate an agumented simulator 
* Evaluation: once the benchmark done, it allow to evaluate the performance with respect to various point of views

![title](img/Benchmarking_scheme_v2.png)

## Benchmark 1 : Risk assessment through contingency screening
The first power grid benchmark tackles the step of risk assessment in power grids. The problem is to anticipate potential threats on the power grid (several hours ahead) and warn the operators accordingly. It simulates incidents (aka contingencies) involving various elements of the grid (such as the disconnection of a line/production unit), one by one. For each contingency, a risk (weakness of the grid) is detected when overloads are detected by the simulation engine on some lines. On a real grid, this scenario means running several dozens of thousands of simulations, thereby, computation time is critical, especially since this risk assessment is refreshed every 15 minutes. In this benchmark, the main physical variable we are interested in is the value of electric current in the lines (in amperes), because an overload is detected if this value exceeds a line-dependent threshold.

First we need to create the associated benchmark:

In [3]:
from lips.benchmark import PowerGridBenchmark
benchmark1 = PowerGridBenchmark(path_benchmark=path_benchmark,
                                benchmark_name="Benchmark1",
                                load_data_set=False,
                                path_config=None, # opt for default config path (benchmark/config.ini)
                                log_path=os.path.abspath(os.path.join("lips","logger","logs.log")) 
                               )

2022-01-07 15:17:31.711672: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-01-07 15:17:31.711733: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Then we need to generate the data, note that this step needs to be done only once (by the person in charge of releasing the benchmark).

People that want to use this benchmark are after free to load the data (see the next notebook for an example)

**NB** To generate the datasets, you need extra libraries (for example `grid2op`, `lightsim2grid` or `leap_net`). These libraries are not required to use this platform once the data are generated.

**NB** This process takes a lot of time...

In [4]:
benchmark1.generate(nb_sample_train=int(1e5),
                    nb_sample_val=int(1e4),
                    nb_sample_test=int(1e4),
                    nb_sample_test_ood_topo=int(1e4),
                   )

test_ood_topo: 100%|████████████████████████████████████████████████████████████████████████████████| 10000/10000 [07:14<00:00, 23.03it/s]


## Benchmark 2 : Remedial action search
Once risky contingencies are identified, we need to explore possible solutions (aka "remedial actions") to recommend suitable solutions to the grid operator. In this benchmark, a solution consists in predefined topological change on the grid. It is successful if the simulation run on this modified grid alleviates the previous overflow without generating any new problem. In this case, the computation space in practice is significantly smaller compared to benchmark 1 (only small subsets of remedial actions to test over in most cases, compared to every contingency simulations). We here target  10-100 simulation batches whatever the grid size. This only depends on the number of preselected action candidates by the operator for a given risk. This benchmark includes the prediction of a few more physical variables: active power flows $p^\ell$ (in MW), current $a^\ell$ (in A) voltages $v_{k}$ (in V) at both sides of each powerline. Although this is not sufficient to fully describe the grid from a physical point of view, it is enough to give the operator some confident insights about the underlying physical behavior. For instance, is an overload somehow rather local and related to neighboring voltage drop phenomena or possibly long-distance considering power flow transmission from large power plants?

Firt we need to create the associated benchmark:

In [5]:
benchmark2 = PowerGridBenchmark(path_benchmark=path_benchmark,
                                benchmark_name="Benchmark2",
                                load_data_set=False,
                                path_config=None, # opt for default config path (benchmark/config.ini)
                                log_path=os.path.abspath(os.path.join("lips","logger","logs.log")) 
                               )

In [6]:
benchmark2.generate(nb_sample_train=int(1e5),
                    nb_sample_val=int(1e4),
                    nb_sample_test=int(1e4),
                    nb_sample_test_ood_topo=int(1e4),
                   )

test_ood_topo: 100%|████████████████████████████████████████████████████████████████████████████████| 10000/10000 [10:11<00:00, 16.36it/s]


## Benchmark 3 : Validation of decision
Once preferred solutions have been selected by the operator to alleviate an overflow, a last detailed simulation is run and studied more in depth by the operator, prior to apply the remedial actions on the actual power grid. The purpose of this operation is to ensure that no unforeseen event may make the grid collapse. In this application, the quality of prediction is decisive, to ensure that the adopted interventions will not violate the consistency within the network; therefore the surrogate simulator shall be nearly as good as the numerical solver. Here the main goal is to ensure the accuracy and consistency of the solution for 1-10 simulations. All physical variables of the physical problem shall be predicted: currents $a^\ell$, active power flows $p^\ell$, voltages $v_{k}$, reactive power flows $q^\ell$, angles $\theta_k$.

<span style="color:red">This benchmark is under progress</span>

In [12]:
benchmark3 = PowerGridBenchmark(path_benchmark=path_benchmark,
                                benchmark_name="Benchmark3",
                                load_data_set=False,
                                path_config=None, # opt for default config path (benchmark/config.ini)
                                log_path=os.path.abspath(os.path.join("lips","logger","logs.log")) 
                               )

In [14]:
benchmark3.generate(nb_sample_train=int(1e5),
                    nb_sample_val=int(1e4),
                    nb_sample_test=int(1e4),
                    nb_sample_test_ood_topo=int(1e4),
                   )

test_ood_topo: 100%|████████████████████████████████████████████████████████████████████████████████| 10000/10000 [05:24<00:00, 30.85it/s]
